This code is used for scraping text data for document classification and building training corpora.

In [28]:
import requests
from bs4 import BeautifulSoup, SoupStrainer
import pickle

##Assumed unreliable sources

In [ ]:
#http://www.homeoint.org    #Scrapes Homeopathy site
for i in range(40):
    url = 'http://www.homeoint.org/hompath/articles/' + str(i) + '.html'
    r = requests.get(url)
    if r.status_code != 404:
        try:
            soup = BeautifulSoup(r.text, 'html.parser')
            titles = soup.find_all('title')
            the_title = titles[0].text
            text = soup.find_all('br')
            content = text[0].text
            with open('/users/markregalla/desktop/metis/Project4/HomeoArticles/page_' 
                      + str(i) +'.txt', 'w') as myfile:
                #myfile.write((the_title + '\n' + content).encode("UTF-8"))
                myfile.write((the_title + content).encode("UTF-8"))
            myfile.close
        except:
            pass
    sys.stdout.write("\r" + str(i))   

In [ ]:
#http://www.homeopathycenter.org/   #doesn't take many requests to jam up server
url = 'http://www.homeopathycenter.org/find-remedy/a'# + letters[0]
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
text = soup.find_all('p')
soup.text

In [ ]:
letters = [string.ascii_lowercase]
letters

##Assumed reliable sources

####Mayo Clinic

In [ ]:
#Mayo Clinic Drugs and supplements articles
url = 'http://www.mayoclinic.org/drugs-supplements'
r = requests.get(url)
print r.status_code
soup = BeautifulSoup(r.text, 'html.parser')
i = 0
ref_list = []
for link in soup.find_all('a'):
    if i in range(108, 149):
        ref_list.append(link['href'])
    i += 1
k = 0
for link in ref_list:
    r = requests.get('http://www.mayoclinic.org/' + link)
    soup = BeautifulSoup(r.text, 'html.parser')
    texts = soup.find_all('p')
    text_list = []
    i = 0 
    for text in texts:
        text_list.append(text.text)
        i += 1
    
    document = ''
    for element in text_list[3:len(texts) - 8]:
        document = document + str(element.encode("UTF-8"))
    try:
        with open('/users/markregalla/desktop/metis/Project4/MayoArticles/page_' 
                          + str(k) +'.txt', 'w') as myfile:
            myfile.write(document.encode("UTF-8"))
        myfile.close
    except:
        pass
    sys.stdout.write("\r" + str(k)) 
    k += 1

####Merck Manual

In [2]:
#Request main page of Merck Professional
merckurl = 'https://www.merckmanuals.com/professional'
r = requests.get(merckurl)
soup = BeautifulSoup(r.text, 'html.parser')

In [3]:
#Generate list of medical topics for scraping
k = 1
medtopiclist = []
for i in soup.find_all('a', href=True):
    #print k, '', i['href']
    if k in range(6,30):
        medtopiclist.append(i['href'])
    k += 1
medtopiclist

[u'/professional/cardiovascular-disorders',
 u'/professional/clinical-pharmacology',
 u'/professional/critical-care-medicine',
 u'/professional/dental-disorders',
 u'/professional/dermatologic-disorders',
 u'/professional/ear,-nose,-and-throat-disorders',
 u'/professional/endocrine-and-metabolic-disorders',
 u'/professional/eye-disorders',
 u'/professional/gastrointestinal-disorders',
 u'/professional/genitourinary-disorders',
 u'/professional/geriatrics',
 u'/professional/gynecology-and-obstetrics',
 u'/professional/hematology-and-oncology',
 u'/professional/hepatic-and-biliary-disorders',
 u'/professional/immunology;-allergic-disorders',
 u'/professional/infectious-diseases',
 u'/professional/injuries;-poisoning',
 u'/professional/musculoskeletal-and-connective-tissue-disorders',
 u'/professional/neurologic-disorders',
 u'/professional/nutritional-disorders',
 u'/professional/pediatrics',
 u'/professional/psychiatric-disorders',
 u'/professional/pulmonary-disorders',
 u'/professional

In [22]:
#Request each website from medical topics list

merckurl_1 = 'https://www.merckmanuals.com' + medtopiclist[0]
r = requests.get(merckurl_1)
soup_1 = BeautifulSoup(r.text, 'html.parser')

In [23]:
#Loop through medical topic list and find links on those pages

k = 1
for i in soup_1.find_all('a', href=True):
    print k, '', i['href']
    k += 1

1  #
2  #
3  /home
4  /
5  /professional
6  /professional/cardiovascular-disorders
7  /professional/clinical-pharmacology
8  /professional/critical-care-medicine
9  /professional/dental-disorders
10  /professional/dermatologic-disorders
11  /professional/ear,-nose,-and-throat-disorders
12  /professional/endocrine-and-metabolic-disorders
13  /professional/eye-disorders
14  /professional/gastrointestinal-disorders
15  /professional/genitourinary-disorders
16  /professional/geriatrics
17  /professional/gynecology-and-obstetrics
18  /professional/hematology-and-oncology
19  /professional/hepatic-and-biliary-disorders
20  /professional/immunology;-allergic-disorders
21  /professional/infectious-diseases
22  /professional/injuries;-poisoning
23  /professional/musculoskeletal-and-connective-tissue-disorders
24  /professional/neurologic-disorders
25  /professional/nutritional-disorders
26  /professional/pediatrics
27  /professional/psychiatric-disorders
28  /professional/pulmonary-disorders
29

In [25]:
#Get sublist of links on each medical topic page
#if medical topic is in url and it is not an old version
link_iter = 0
scraper_list = []
for link in medtopiclist:
    k = 1
    merckurl_1 = 'https://www.merckmanuals.com' + medtopiclist[link_iter]
    r = requests.get(merckurl_1)
    soup_1 = BeautifulSoup(r.text, 'html.parser')
    for i in soup_1.find_all('a', href=True):
        if k >= 126:
            if str(medtopiclist[link_iter][14:]) in str(i):
                if '#v' not in str(i):
                    scraper_list.append(i['href'])
        k += 1
    link_iter += 1

print len(scraper_list)
scraper_list = set(scraper_list)
print len(scraper_list)
scraper_list

5311
2457


{u'/professional/cardiovascular-disorders/lymphatic-disorders/overview-of-the-lymphatic-system',
 u'/professional/psychiatric-disorders/somatic-symptom-and-related-disorders/overview-of-somatization',
 u'/professional/pediatrics/congenital-renal-and-genitourinary-anomalies/bladder-anomalies',
 u'/professional/dermatologic-disorders/benign-skin-tumors,-growths,-and-vascular-lesions/nevus-araneus',
 u'/professional/pulmonary-disorders/symptoms-of-pulmonary-disorders/stridor',
 u'/professional/injuries;-poisoning/thoracic-trauma/aortic-disruption-(traumatic)',
 u'/professional/infectious-diseases/bacteria-and-antibacterial-drugs/trimethoprim-and-sulfamethoxazole',
 u'/professional/gastrointestinal-disorders/bezoars-and-foreign-bodies/foreign-bodies-in-the-gi-tract',
 u'/professional/hematology-and-oncology/thrombocytopenia-and-platelet-dysfunction/hereditary-intrinsic-platelet-disorders',
 u'/professional/gastrointestinal-disorders/inflammatory-bowel-disease-(ibd)/overview-of-inflammatory

###Experimental total scraper

In [ ]:
#Function takes website request text and scrapes all links on that site
#Not functioning properly yet
def Scrape_Whole_Site(url):
    link_list = []
    master_list = []
    successes = 0
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    master_links = soup.find_all('a')
    
    for link in master_links:
        link_list.append(link['href'])
        master_list.append(link['href'])
    
    #for link in link_list:
        #print link
        #print link[:len(url)]
    
    for link in link_list:
        try:
            sub_request = requests.get(link)
        except:
            pass
        sub_soup = BeautifulSoup(sub_request.text, 'html.parser')
        sub_links = sub_soup.find_all('a')
        for sub_link in sub_links:
            try:
                if sub_link['href'][:len(url)] == url:
                    if sub_link['href'] not in master_list:
                        #print sub_link['href']
                        master_list.append(sub_link['href'])
                        successes += 1
                        sys.stdout.write("\r" + 'Sucesses: ' + str(successes))   
            except:
                pass
        
    first_len = len(master_list)
    set_list = set(master_list)
    set_len = len(master_list)
    
    return first_len, set_len
    
first_len, set_len = Scrape_Whole_Site(url)
print ' '
print 'Length: ' + str(first_len)
print 'Set len: ' + str(set_len)
print 'Done'
for url in master_list:
    print url

##Wikipedia scraping for Doc2Vec training
Runs them through training model

Wikipedia API guide:
https://en.wikipedia.org/wiki/Special:ApiSandbox#action=query&prop=info&format=json&inprop=url&pageids=1000005

In [ ]:
%%time
i = 259987          #i = 259987   (12-5-15 1:43 pm)
counter = 50000   #counter = 50000 (12-5-15 1:43 pm)
good_id_list = [] #be sure only to initialize as empty list the first time
#id_list = []      #use new list if you want to preserve good_id_list in memory

while counter <= 70000:   #number is limit of number of page ids to get
    scrape_worked = False
    #if i == 1000:
    #    print 'went on too long'
    #    break
    page_id = 1000000 + i
    
    #try to scrape webpage.  Ignore if bad page error is returned from wikipedia
    try:
        r = requests.get('https://en.wikipedia.org/w/api.php?action=query&prop=info\
                &format=json&inprop=url&pageids=' + str(page_id))
        rtext = r.text
        jsontext = json.loads(rtext)
        if jsontext['query']['pages'][str(page_id)]['ns'] == 0:
            scrape_worked = True
    except:
        i += 1    
    
    if scrape_worked == True:
        article = requests.get('https://en.wikipedia.org/?curid=' + str(page_id))
        soup = BeautifulSoup(article.text, 'html.parser')
        paragraphs = soup.findAll('p')
        if len(paragraphs) >= 10:
            good_id_list.append(page_id)
            #id_list.append(page_id)    #for preservation of good_id_list
            corpus = page_read(soup)
            corpus = [corpus]
            corpus = cleanText(corpus)
            #write each wikipedia article to its own file
            with open("/users/markregalla/desktop/metis/Project4/WikiArticles/page_" 
                      + str(page_id) + ".txt", "w") as myfile:
                myfile.write(corpus.encode("UTF-8"))
            myfile.close
            counter += 1
    i += 1
    
    #prints iterators as a status
    sys.stdout.write("\r" + 'Attempts: ' + str(i) + '  Sucesses: ' + str(counter))   

print '\n' + str(len(good_id_list))
#print '\n' + str(len(id_list))

##Pickling

In [29]:
#dump list of good id nos into pickle file
with open('Merck_scrape_list.pkl', 'w') as picklefile:
    pickle.dump(scraper_list, picklefile)